<a href="https://colab.research.google.com/github/Abinaya-J/Udacity_SQL_for_Data_Analysis/blob/master/Lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Subqueries
#We want to find the average number of events for each day for each channel. The first table will provide us the number of events for each day and channel, 
#and then we will need to average these values together using a second query.
SELECT channel,AVG(event_count) avg_event_count
FROM (
SELECT DATE_TRUNC('day',occurred_at) day_occured,channel,COUNT(*) event_count
FROM web_events
GROUP BY 1,2
) sub
GROUP BY 1

In [ ]:
#Pull the first month/year combo from the orders table
#Find orders that took place in the same year/month combo and the average quantities of each type of paper
SELECT AVG(standard_qty) avg_std, AVG(gloss_qty) avg_gls, AVG(poster_qty) avg_pst,SUM(total_amt_usd)
FROM orders
WHERE DATE_TRUNC('month',occurred_at)= 
     (SELECT DATE_TRUNC('month',MIN(occurred_at)) month_occured
FROM orders)

In [ ]:
#More on Subqueries
#Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
SELECT sub3.rep_name,sub3.region_name,sub3.total_amt
FROM(SELECT region_name,MAX(total_amt) total_max_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
          FROM sales_reps s
          JOIN accounts a
          ON a.sales_rep_id = s.id
          JOIN orders o
          ON o.account_id = a.id
          JOIN region r
          ON r.id = s.region_id
          GROUP BY 1, 2) sub1                         
          GROUP BY 1)
JOIN(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1, 2
     ORDER BY 3 DESC)sub3
ON sub3.region_name = sub2.region_name AND sub3.total_amt = sub2.total_max_amt;

#For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?
SELECT r.name,COUNT(o.total) total_orders
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name
HAVING SUM(o.total_amt_usd)=(SELECT MAX(total_amt)
                            FROM(SELECT r.name region_name, SUM(o.total_amt_usd) total_amt
                                FROM sales_reps s
                                JOIN accounts a
                                ON a.sales_rep_id = s.id
                                JOIN orders o
                                ON o.account_id = a.id
                                JOIN region r
                                ON r.id = s.region_id
                                GROUP BY 1) );